In [1]:
### Note, crispr pool
# these libs were to check reproducibility in making pools

In [2]:
import gzip
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path
import regex as re

def open_gzip_optional(fname,mode="rt"):
    if str(fname).endswith("gz"):
        return gzip.open(fname,mode)
    elif str(fname).endswith("bz2"):
        return bz2.open(fname,mode)
    else:
        return open(fname,mode)
            
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]])

#rootdir = Path("/corgi/otherdataset/ellenbushell/phit/")
rootdir = Path("/corgi/otherdataset/ellenbushell/2023march_pools/")  
fastqdir = rootdir / "fastq"

################ Read barcode collection. barcode = sgRNA
#import pandas as pd
bc_file = pd.read_csv("/corgi/otherdataset/ellenbushell/2023march_pools/bc.csv",sep="\t")
bc_file.columns = ["mutant","barcode"]
dict_bc2mutant = dict(zip(bc_file["barcode"],bc_file["mutant"]))



################ Prepare parser
flank_regex = re.compile("ATTATT"+"(\w{18,30})"+"GTTT")

################ Process all files
#sample_dict = {}
list_samples = []
list_counts = []
for seq_file in fastqdir.iterdir():
    if str(seq_file).endswith("fastq.gz"):
        
        #if len(list_samples)>5:
        #    break


        ####### Extract sample name
        #P28011_1172_S11_L001_R1_001.fastq.gz
        # => P28011_1172
        #sample = format_filename(seq_file)
        sample = seq_file.name.split("_S")[0]
        
        #Empty counts dictionary
        count_dict = dict(zip(list(bc_file["mutant"]), [0]*bc_file.shape[0]))
        count_dict["_other"] = 0 
        count_dict["_err"] = 0 
        
        #Reverse complement if read2
        do_revcomp = ("_R2_" in str(seq_file))
        
        if "_R1_" in seq_file.name:
            sample = sample+":R1"
        else:
            sample = sample+":R2"
        print(sample)
        
        
        #For all lines
        with open_gzip_optional(seq_file) as f1:
            it1 = iter(f1)
            count = 0
            while True:
                try:
                    
                    #Read one entry in FASTQ
                    next(it1)
                    theseq = next(it1).strip()
                    next(it1)
                    next(it1)
                    
                    if do_revcomp:
                        theseq = reverse_complement(theseq)

                    #print("> "+theseq)
                        
                    #Parse one entry
                    try:
                        
                        putative_barcode = re.search(flank_regex, theseq)[1]
                        #print(putative_barcode)
                        
                        putative_barcode = putative_barcode  #for this input file
                        
                        if putative_barcode in dict_bc2mutant.keys():
                            count_dict[dict_bc2mutant[putative_barcode]] += 1
                        else:
                            count_dict["_other"] += 1
                            #print(str(count)+" not in dict "+putative_barcode)

                    except TypeError:
                        count_dict["_err"] += 1
                        #print(str(count)+" type err: "+theseq)
                    
                    count=count+1

                except StopIteration:
                    break
        
        #Turn into an array, add to the list
        list_samples.append(sample)
        list_counts.append(
            np.array([count_dict["_other"], count_dict["_err"]]+[count_dict[i] for i in bc_file["mutant"]])
        )
        
        

### Turn it all into one matrix
df = pd.DataFrame(
    np.vstack(list_counts).T,
    columns = list_samples, 
    index = ["_other","_err"]+list(bc_file["mutant"]))

P28657_1017:R1
P28657_1017:R2
P28657_1018:R1
P28657_1018:R2
P28657_1019:R1
P28657_1019:R2
P28657_1020:R1
P28657_1020:R2
P28657_1021:R1
P28657_1021:R2
P28657_1022:R1
P28657_1022:R2
P28657_1023:R1
P28657_1023:R2
P28657_1024:R1
P28657_1024:R2
P28657_1025:R1
P28657_1025:R2


In [3]:
df

,P28657_1017:R1,P28657_1017:R2,P28657_1018:R1,P28657_1018:R2,P28657_1019:R1,P28657_1019:R2,P28657_1020:R1,P28657_1020:R2,P28657_1021:R1,P28657_1021:R2,P28657_1022:R1,P28657_1022:R2,P28657_1023:R1,P28657_1023:R2,P28657_1024:R1,P28657_1024:R2,P28657_1025:R1,P28657_1025:R2
_other,43954,46555,37468,40724,41475,44564,20695,23702,38886,48810,41195,47803,30152,28864,22679,22948,32050,32631
_err,7480,5940,6393,5443,7110,5708,3846,3297,7666,6312,7594,6015,5860,4033,4654,3463,6812,4758
PBANKA_0211000.1,19140,20431,16331,17316,17409,18601,2764,2925,2477,2606,3429,3598,4205,4431,21,22,3185,3361
PBANKA_0211000.2,47810,45758,46622,44472,50979,48703,27151,25717,43039,40142,15951,14958,11952,11461,5334,5084,5875,5555
PBANKA_0515000.1,8861,8732,7375,7201,5935,5782,2940,2844,1141,1075,4222,4071,1531,1501,7237,7056,30224,29564
PBANKA_0706400.2,67932,67612,77040,76680,58266,58046,83563,82767,153161,149389,179519,176519,6911,6905,8895,8818,50770,50279
PBANKA_0706400.1,55165,56402,41212,41989,40348,41287,27879,28200,39183,39277,46987,47548,8628,8789,10235,10467,6,5
PBANKA_0933700.2,2,2,8,10,407,414,2,3,305,298,1432,1434,1,1,3827,3848,0,0
PBANKA_1013300.1,32809,32406,22812,22356,28389,27855,7264,7094,6922,6546,3473,3315,1129,1116,2733,2680,268,262
PBANKA_1013300.2,59778,59123,66594,65679,65566,64948,10911,10655,19030,18446,27252,26519,14541,14481,12669,12430,19574,19355


In [4]:
#Check how well it worked
df[0:2]/df.sum(axis=0)

#df[-1]/df.sum(axis=0)



,P28657_1017:R1,P28657_1017:R2,P28657_1018:R1,P28657_1018:R2,P28657_1019:R1,P28657_1019:R2,P28657_1020:R1,P28657_1020:R2,P28657_1021:R1,P28657_1021:R2,P28657_1022:R1,P28657_1022:R2,P28657_1023:R1,P28657_1023:R2,P28657_1024:R1,P28657_1024:R2,P28657_1025:R1,P28657_1025:R2
_other,0.128112,0.135693,0.116201,0.126299,0.131226,0.140999,0.099012,0.113398,0.110787,0.139061,0.111053,0.128867,0.105004,0.100519,0.093539,0.094648,0.098139,0.099918
_err,0.021802,0.017313,0.019827,0.016881,0.022496,0.018060,0.018401,0.015774,0.021841,0.017983,0.020472,0.016215,0.020407,0.014045,0.019195,0.014283,0.020859,0.014569


In [5]:
df.iloc[ -1]/df.sum(axis=0)


P28657_1017:R1    0.000009
P28657_1017:R2    0.000015
P28657_1018:R1    0.000006
P28657_1018:R2    0.000006
P28657_1019:R1    0.000016
P28657_1019:R2    0.000019
P28657_1020:R1    0.000000
P28657_1020:R2    0.000000
P28657_1021:R1    0.000003
P28657_1021:R2    0.000011
P28657_1022:R1    0.000008
P28657_1022:R2    0.000003
P28657_1023:R1    0.000000
P28657_1023:R2    0.000000
P28657_1024:R1    0.000004
P28657_1024:R2    0.000008
P28657_1025:R1    0.000006
P28657_1025:R2    0.000012
dtype: float64

In [6]:
df.to_csv(rootdir / "counts.csv")

#rootdir = Path("/corgi/otherdataset/ellenbushell/P28011/")
